# Chapter 9: Spark MLlib and ML

In this notebook, we will see the main capabilities of Spark MLlib and ML.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark-MLlib-ML").master("local[*]").getOrCreate()
sc = spark.sparkContext

## Working with MLlilb

In this section, we will focus on MLlib

In [2]:
from pyspark.mllib.linalg import DenseVector, SparseVector
from pyspark.mllib.feature import HashingTF, Word2Vec, IDF, StandardScaler, ChiSqSelector
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import numpy as np
from random import randint

### MLlib Feature Encoding and Data Preparation

#### Working with Spark Vectors

We can create Dense Vectors, Sparse Vectors and Labeled Points

In [3]:
dense_vector = DenseVector(np.array([1,2,3]))

In [4]:
dense_vector

DenseVector([1.0, 2.0, 3.0])

In [5]:
sparse_vector = SparseVector(4, {0:1.5, 2:3})

In [6]:
sparse_vector

SparseVector(4, {0: 1.5, 2: 3.0})

In [7]:
labeled_point = LabeledPoint(1, dense_vector)

In [8]:
labeled_point

LabeledPoint(1.0, [1.0,2.0,3.0])

#### Preparing Textual Data

We can also prepare text data using some in-built data transformations capabilities already included in MLlib. We first prepare some text data about Spam and Non-Spam emails.

In [9]:
ini_data = spark.read.csv("../data/spam.csv", header=True)
ini_data_rdd = ini_data.select(["label", "text"]).rdd.filter(lambda row: (isinstance(row.label, str) and isinstance(row.text, str)))

In [10]:
ini_data_rdd.count()

5573

In [11]:
ini_data_rdd.take(1)

[Row(label='ham', text='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')]

In [12]:
text_rdd = ini_data_rdd.map(lambda x: x.text)

Now we use the `HashingTF` transformer.

In [13]:
def hasing_TF(text_rdd):
    """
    Transforms an input RDD of text using the Hashing TF transformer
    
    :input text_rdd: input RDD
    :return: transformed RDD
    """
    tokenizer = HashingTF()
    text_tokenized = text_rdd.map(lambda text: text.split(" "))
    return tokenizer.transform(text_tokenized)

In [14]:
text_rdd.take(1)

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...']

In [15]:
hash_text = hasing_TF(text_rdd)

In [16]:
hash_text.take(1)

[SparseVector(1048576, {79172: 1.0, 244892: 1.0, 296409: 1.0, 312753: 1.0, 384022: 1.0, 407924: 1.0, 414297: 1.0, 442668: 1.0, 627241: 1.0, 639697: 1.0, 799074: 1.0, 856522: 1.0, 897134: 1.0, 901549: 1.0, 968035: 1.0, 988036: 1.0, 997716: 1.0, 1015964: 1.0, 1033917: 1.0, 1044354: 1.0})]

In [17]:
def hasing_TF_with_text(text_rdd):
    """
    Transforms an input RDD of text using the Hashing TF transformer
    keeping also the original text
    
    :input text_rdd: input RDD
    :return: transformed RDD
    """
    tokenizer = HashingTF()
    return text_rdd.map(lambda text: (text, tokenizer.transform(text.split(" "))))

In [18]:
hash_text_perserving = hasing_TF_with_text(text_rdd)

In [19]:
hash_text_perserving.take(1)

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  SparseVector(1048576, {79172: 1.0, 244892: 1.0, 296409: 1.0, 312753: 1.0, 384022: 1.0, 407924: 1.0, 414297: 1.0, 442668: 1.0, 627241: 1.0, 639697: 1.0, 799074: 1.0, 856522: 1.0, 897134: 1.0, 901549: 1.0, 968035: 1.0, 988036: 1.0, 997716: 1.0, 1015964: 1.0, 1033917: 1.0, 1044354: 1.0}))]

We can now use the `Word2Vec` transformer.

In [20]:
text_tokenized = text_rdd.map(lambda text: text.split(" "))
word2vec_trformer = Word2Vec().fit(text_tokenized)

In [21]:
word2vec_trformer.transform("great")

DenseVector([0.0214, 0.0091, -0.0685, 0.0484, -0.0635, -0.014, -0.1073, 0.0733, -0.0456, -0.0559, -0.0246, 0.0231, -0.005, 0.0689, 0.0356, -0.0237, -0.0297, -0.0312, -0.1064, -0.0312, -0.0419, -0.0526, 0.039, 0.0809, 0.0011, -0.0741, -0.0025, 0.0244, -0.0908, 0.0746, -0.035, 0.0464, -0.0426, 0.004, 0.0525, 0.0713, -0.0321, -0.0063, 0.0798, -0.0541, 0.0303, -0.0136, -0.0675, -0.0585, -0.016, 0.0131, -0.0729, -0.0461, -0.0227, 0.0301, 0.0278, -0.0161, -0.019, 0.0209, -0.0354, -0.0247, -0.0088, 0.0109, -0.021, -0.0587, -0.0034, -0.0333, 0.0591, -0.0112, 0.0712, 0.0155, 0.0203, -0.0088, -0.0207, 0.1669, -0.0268, 0.0282, -0.0365, -0.0001, -0.0326, -0.0124, -0.0468, -0.0332, -0.0206, -0.005, 0.033, 0.0429, -0.0292, 0.0625, 0.0093, 0.0847, -0.0263, -0.035, 0.1426, -0.0106, 0.0819, 0.1647, -0.0244, -0.0121, 0.03, -0.0592, 0.0011, -0.0645, -0.015, -0.0845])

In [22]:
word2vec_trformer.transform("Free")

DenseVector([0.0113, -0.0021, -0.0235, 0.0119, 0.0046, 0.0445, -0.0227, 0.0225, 0.0213, 0.0038, 0.038, 0.0114, 0.0031, 0.0007, -0.0047, -0.0152, 0.0063, 0.0075, -0.0435, 0.0189, 0.018, -0.0127, 0.0226, 0.0255, 0.0116, -0.0141, 0.0172, 0.0183, -0.0344, 0.0133, 0.0032, 0.0066, 0.0004, -0.0111, 0.0135, 0.0069, -0.0006, -0.0153, 0.0055, 0.0038, -0.0063, -0.017, -0.0496, -0.0017, 0.0529, -0.0248, -0.0282, -0.0114, 0.0076, 0.0195, 0.0124, 0.0183, 0.0061, 0.013, -0.0146, -0.0234, 0.0096, 0.0088, 0.0162, 0.0059, -0.0129, 0.0101, 0.0484, 0.0004, 0.0229, 0.0075, -0.004, -0.003, -0.0059, 0.0564, -0.0007, 0.0109, -0.0068, -0.0093, 0.0057, 0.0186, 0.0004, 0.0053, -0.0059, -0.0072, -0.0029, 0.0344, 0.0158, 0.0013, 0.0028, 0.0219, 0.0096, 0.0324, 0.036, -0.0058, -0.0068, -0.0044, 0.0059, -0.0126, 0.0226, -0.0233, 0.0221, -0.0021, 0.0126, -0.044])

#### Preparing Data for Supervised Learning

In [23]:
tf = HashingTF(1000)
tf_vectors = tf.transform(text_rdd)
idf = IDF()
idf_model = idf.fit(tf_vectors)

In [24]:
spam_text = ini_data_rdd.filter(lambda row: row.label == "spam").map(lambda row: row.text.split(" "))
gen_text = ini_data_rdd.filter(lambda row: row.label != "spam").map(lambda row: row.text.split(" "))

In [25]:
spam_points = idf_model.transform(tf.transform(spam_text)).map(lambda x: LabeledPoint(1, x))
gen_points = idf_model.transform(tf.transform(gen_text)).map(lambda x: LabeledPoint(0, x))

In [26]:
ml_data_ini = spam_points.union(gen_points)

In [27]:
ml_data = ml_data_ini.map(lambda row: (randint(0,100), row)).sortByKey().map(lambda row: row[1])

In [28]:
ml_data_train, ml_data_test = ml_data.randomSplit(weights = [0.8, 0.2])

In [29]:
ml_data_train.cache()
ml_data_test.cache()

PythonRDD[54] at RDD at PythonRDD.scala:49

#### Feature Scaling and Selection

It is useful sometimes for the ML algorithms to scale that data.

`StandardScaler()` --> to scale numerical data

In [30]:
from pyspark.mllib.feature import StandardScaler
std_scaler = StandardScaler()
std_scaler_model = std_scaler.fit(ml_data.map(lambda lpoint: lpoint.features))

In [31]:
train_label = ml_data_train.map(lambda lpoint: lpoint.label)
test_label = ml_data_test.map(lambda lpoint: lpoint.label)

In [32]:
ml_data_train_scl = train_label.zip(std_scaler_model.transform(ml_data_train.map(lambda lpoint: lpoint.features)))\
.map(lambda x: LabeledPoint(x[0], x[1]))

ml_data_test_scl = test_label.zip(std_scaler_model.transform(ml_data_test.map(lambda lpoint: lpoint.features)))\
.map(lambda x: LabeledPoint(x[0], x[1]))

In [33]:
ml_data_train_scl.take(1)

[LabeledPoint(1.0, (1000,[4,52,147,246,272,276,280,289,303,311,318,321,345,365,417,427,478,530,562,567,695,770,781,786,940,999],[8.51175129892,2.47883625125,2.72403263804,10.4969379283,14.1423077767,8.56298964927,10.4933640717,1.86079465498,9.84937323593,2.13449853856,4.54232967283,1.4744202717,7.88135134191,1.4119082138,13.4442310562,8.11210138222,17.6226733461,7.76209066016,11.047722177,12.8363865401,3.75100403532,3.67306602891,9.68903342154,8.79126951606,7.12532988342,11.5617507079]))]

In [34]:
ml_data_test_scl.take(1)

[LabeledPoint(1.0, (1000,[49,69,79,105,161,177,212,266,284,287,300,320,513,704,719,740,862,870,872,950],[17.6226733461,4.1679073928,11.0516323062,17.1431514526,23.6819104978,9.76643107601,11.6928010764,13.6652319833,8.85130603839,6.23128041373,3.1673723595,5.12026937089,20.0369283876,15.5969494943,10.7070843704,10.8254506863,5.55891391747,2.34265780804,4.46649798562,13.6652319833]))]

`ChiSqSelector` --> to select the most relevant features

In [35]:
selector = ChiSqSelector(100)

In [36]:
selector_model = selector.fit(ml_data)

In [37]:
selector_model

In [38]:
ml_data_train_sel = train_label.zip(selector_model.transform(ml_data_train.map(lambda lpoint: lpoint.features)))\
.map(lambda x: LabeledPoint(x[0], x[1]))

ml_data_test_sel = test_label.zip(selector_model.transform(ml_data_test.map(lambda lpoint: lpoint.features)))\
.map(lambda x: LabeledPoint(x[0], x[1]))

In [39]:
ml_data_train_sel.take(1)

[LabeledPoint(1.0, (100,[14,25,34,98],[8.62586820804,0.0939834678827,8.62586820804,8.62586820804]))]

In [40]:
ml_data_test_sel.take(1)

[LabeledPoint(1.0, (100,[7,27],[8.62586820804,8.62586820804]))]

### MLlib Model Training

Once we have prepared our data, we can train some models

In [41]:
lr = LogisticRegressionWithLBFGS()
lr_model_raw = lr.train(ml_data_train)
lr_model_scl = lr.train(ml_data_train_scl)

In [42]:
lr_model_raw.weights

DenseVector([-1.9284, 1.2011, -1.6716, -0.1829, -0.8703, 6.2302, -0.83, -5.1025, -2.7163, -3.4091, -6.5753, -1.6253, -2.1025, -3.928, 1.8886, 0.3095, -3.9537, -1.2226, -2.2495, 2.6413, 1.2776, -2.5857, -1.0293, -2.8195, -2.16, -6.7403, 4.0004, -2.2021, -0.7097, -6.0831, -2.2067, 18.4533, 2.3616, 0.8644, -4.5893, 0.2858, -1.9696, -2.3202, -0.3221, 3.1445, -1.5311, 3.772, -4.0671, -2.8515, -1.1902, 9.0124, 3.4538, -5.4371, 2.1061, 0.9165, -5.6358, 3.8224, -2.7222, -5.2017, 4.0691, 2.1829, -3.1017, -1.1124, -1.6781, -0.664, 2.428, 4.0929, -7.0371, -0.716, -2.3151, -0.3477, -3.8444, -2.183, -0.8714, 2.8121, 2.2898, 1.458, 2.3678, -1.1176, 0.3358, -1.7615, 3.1642, -0.776, -2.6373, -0.7602, -0.242, 0.4789, -0.3451, -2.8576, -6.0851, -0.8091, -5.4381, 0.2377, 1.4896, -2.1936, -0.8667, -3.1433, 41.3328, 0.6083, 0.3643, -3.7917, -1.3229, 5.7845, 0.0139, 0.9996, -1.3369, 1.6173, -3.2322, 0.3681, -165.6355, 3.748, -8.1512, -1.9167, -5.686, -2.7126, 3.5066, 9.6959, -0.5487, 1.7329, -2.1194, -3.674

In [43]:
lr_model_scl.weights

DenseVector([-8.465, 0.8185, -1.5299, -0.4801, -0.882, 0.6729, -0.5154, -3.0006, -2.2978, -4.1935, -4.1505, -0.9555, -4.3706, -2.7913, 1.4746, 0.2494, -8.8625, -1.4771, -0.4628, 1.9235, 0.871, -1.6049, -0.7307, -1.658, -1.534, -4.1111, 3.9316, -2.146, -0.8829, -5.1874, -2.6057, 3.1879, 1.5631, 0.7759, -3.342, 0.2546, -2.2017, -4.6477, -0.2164, 1.9186, -1.441, 2.8472, -4.8214, -9.8012, -0.8991, 5.5937, 1.7367, -4.9214, 1.7634, 0.4486, -3.1046, 3.595, -9.4727, -4.6767, 1.8782, 1.6864, -4.2709, -1.7797, -1.2068, -0.4954, 2.477, 2.7893, -5.3118, -0.9741, -3.5961, -0.2941, -2.7996, -2.5769, -0.4923, 5.8199, 2.2918, 1.3087, 2.1931, -1.4814, 0.2731, -1.2832, 2.1564, -0.5062, -3.5999, -0.5933, -0.7499, 0.2972, -0.3292, -5.227, -3.6488, -0.2954, -6.7328, 0.15, 1.3911, -3.4513, -0.7338, -1.9517, 4.359, 0.7039, 0.3216, -6.8324, -1.5334, 1.2051, 0.0162, 0.6518, -1.0207, 6.1883, -1.9361, 0.2614, -4.8511, 1.8859, -3.2594, -1.0604, -2.9332, -1.8228, 1.1453, 4.5008, -0.5316, 1.492, -1.0937, -4.5106, 1

### Predict

Once the model is trained, we can perform predictions.

In [44]:
raw_preds = lr_model_raw.predict(ml_data_test.map(lambda lpoint: lpoint.features))
scl_preds = lr_model_scl.predict(ml_data_test_scl.map(lambda lpoint: lpoint.features))

In [45]:
raw_preds.take(1)

[0]

In [46]:
scl_preds.take(1)

[0]

### Serving and Persistence

Many times, once we train our model, we save it and the load it in oder programs to make predictions. We try first the internal format of Spark, which allows us to save and load a model.

In [47]:
!rm -rf ../data/lr_model_raw
lr_model_raw.save(sc, "../data/lr_model_raw")

In [48]:
lr_model_raw_loaded = LogisticRegressionModel.load(sc, "../data/lr_model_raw")

In [49]:
raw_preds_loaded = lr_model_raw_loaded.predict(ml_data_test.map(lambda lpoint: lpoint.features))

In [50]:
raw_preds_loaded.take(1)

[0]

### Model Evaluation

MLlib includes some functionalities to calculate automatically some metrics of trained ML models. While there are more, here we will evaluate the LR model of the spam classification section using the `BinaryClassificationMetrics` functionality.

In [51]:
ml_data_train.take(1)

[LabeledPoint(1.0, (1000,[4,52,147,246,272,276,280,289,303,311,318,321,345,365,417,427,478,530,562,567,695,770,781,786,940,999],[8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,1.7444569044,0.0939834678827,8.62586820804,0.244953034918,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804,8.62586820804]))]

In [52]:
pred_label_lr = ml_data_test.map(lambda lpoint: (float(lr_model_raw.predict(lpoint.features)), lpoint.label))
metrics_lr = BinaryClassificationMetrics(pred_label_lr)

In [53]:
print("LR model")
print("Area Under PR: {0}".format(metrics_lr.areaUnderPR))
print("Area Under ROC: {0}".format(metrics_lr.areaUnderROC))

LR model
Area Under PR: 0.5384643691760969
Area Under ROC: 0.8440228211243133


## Working with Spark ML

Now, we are going to see some of the capabilities offered by the Spark ML package, which works with DataFrames instead that MLlib that works with RDDs. In particular, we are going to do again the spam classification problem using two Pipelines: one for the data preparation and the other one for the ML model.

### Data Preparation: Data Encoding & Data Cleaning

In [54]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, IDF, SQLTransformer, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.pipeline import Pipeline, PipelineModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [55]:
ini_data.show()

+-----+--------------------+----+----+----+
|label|                text| _c2| _c3| _c4|
+-----+--------------------+----+----+----+
|  ham|Go until jurong p...|null|null|null|
|  ham|Ok lar... Joking ...|null|null|null|
| spam|Free entry in 2 a...|null|null|null|
|  ham|U dun say so earl...|null|null|null|
|  ham|Nah I don't think...|null|null|null|
| spam|FreeMsg Hey there...|null|null|null|
|  ham|Even my brother i...|null|null|null|
|  ham|As per your reque...|null|null|null|
| spam|WINNER!! As a val...|null|null|null|
| spam|Had your mobile 1...|null|null|null|
|  ham|I'm gonna be home...|null|null|null|
| spam|SIX chances to wi...|null|null|null|
| spam|URGENT! You have ...|null|null|null|
|  ham|I've been searchi...|null|null|null|
|  ham|I HAVE A DATE ON ...|null|null|null|
| spam|XXXMobileMovieClu...|null|null|null|
|  ham|Oh k...i'm watchi...|null|null|null|
|  ham|Eh u remember how...|null|null|null|
|  ham|Fine if that��s t...|null|null|null|
| spam|England v Macedon...|null

In [56]:
sql_select = SQLTransformer(statement = "SELECT label, text FROM __THIS__")

In [57]:
sql_filter = SQLTransformer(statement = "SELECT * from __THIS__ WHERE text is not null AND label is not null")

In [58]:
label_indexer = StringIndexer(inputCol="label", outputCol="label_num")

In [59]:
tokenizer = Tokenizer(inputCol = "text", outputCol = "text_token")

In [60]:
count_text = SQLTransformer(statement = "SELECT *, size(text_token) as count FROM __THIS__")

In [61]:
tf = HashingTF(numFeatures = 1000, inputCol = "text_token", outputCol = "text_tf")

In [62]:
idf = IDF(inputCol = "text_tf", outputCol = "text_features")

In [63]:
assember = VectorAssembler(inputCols = ["text_features", "count"],
                           outputCol = "features_raw")

In [64]:
scaler = StandardScaler(inputCol="features_raw", outputCol="features")

In [65]:
etl_pipeline_model = Pipeline(stages=[sql_select, sql_filter, label_indexer, 
                                      tokenizer, count_text, tf, idf, assember, scaler]).fit(ini_data)

In [66]:
ml_data_pipe = etl_pipeline_model.transform(ini_data)

In [67]:
ml_data_pipe.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  ham|(1001,[7,77,150,1...|
|  ham|(1001,[20,316,484...|
| spam|(1001,[30,35,73,1...|
|  ham|(1001,[57,368,372...|
|  ham|(1001,[135,163,32...|
| spam|(1001,[25,36,91,9...|
|  ham|(1001,[18,47,48,5...|
|  ham|(1001,[36,71,92,2...|
| spam|(1001,[39,43,61,7...|
| spam|(1001,[36,73,82,1...|
|  ham|(1001,[26,41,106,...|
| spam|(1001,[15,35,36,4...|
| spam|(1001,[68,73,122,...|
|  ham|(1001,[19,36,39,1...|
|  ham|(1001,[44,82,170,...|
| spam|(1001,[41,43,49,6...|
|  ham|(1001,[275,426,44...|
|  ham|(1001,[80,147,236...|
|  ham|(1001,[159,170,29...|
| spam|(1001,[9,19,45,71...|
+-----+--------------------+
only showing top 20 rows



In [68]:
ml_data_pipe_train, ml_data_pipe_test = ml_data_pipe.randomSplit(weights=[0.8, 0.2])

In [69]:
ml_data_pipe_train.count()

4486

In [70]:
ml_data_pipe_test.count()

1087

#### Spark ML Models

Once we have our data cleaned and encoded, we can now train a ML models.

In [71]:
lr = LogisticRegression(featuresCol="features", labelCol="label_num", regParam=0.1)

In [72]:
lr_pipeline = Pipeline(stages=[lr])

In [73]:
lr_pipeline_model = lr_pipeline.fit(ml_data_pipe_train)

As in any other pipeline, we can access to one of the steps, and its metadata. Here we get the coefficientMatrix of the trained LR model.

In [74]:
lr_pipeline_model.stages[0].coefficientMatrix

DenseMatrix(3, 1001, [0.0226, 0.0017, -0.003, 0.0209, 0.0078, -0.0099, 0.0171, 0.029, ..., -0.0005, -0.0001, -0.0002, -0.0003, -0.0003, -0.0002, -0.0002, -0.001], 1)

#### Data Persistence and Spark ML

We can save and load our pipelines (including both data transformers and ML algorithms).

In [75]:
!rm -rf ../data/etl_pipeline_model
!rm -rf ../data/lr_pipeline_model
etl_pipeline_model.save("../data/etl_pipeline_model")
lr_pipeline_model.save("../data/lr_pipeline_model")

In [76]:
etl_pipeline_load = PipelineModel.load("../data/etl_pipeline_model")
lr_pipeline_load = PipelineModel.load("../data/lr_pipeline_model")

In [77]:
predictions = lr_pipeline_load.transform(etl_pipeline_load.transform(ini_data))

In [78]:
predictions.select("label_num", "prediction").show()

+---------+----------+
|label_num|prediction|
+---------+----------+
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
+---------+----------+
only showing top 20 rows



#### Automated Model Selection: Parameter Search

Spark ML offers some functionalities to perform hiperparameter tunning on ML models. Let's check our previous problem testing different regularization parameters.

In [79]:
lr = LogisticRegression(featuresCol="features", labelCol="label_num", regParam=0.1)

In [80]:
estimator_pipeline = Pipeline(stages=[lr])

In [81]:
param_grid = ParamGridBuilder().addGrid(LogisticRegression.regParam, [0.1, 0.01, 0.05]).build()

In [82]:
evaluator = BinaryClassificationEvaluator(labelCol="label_num", rawPredictionCol="prediction")

In [83]:
crossval = CrossValidator(estimator=estimator_pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=3)

In [84]:
cv_model = crossval.fit(dataset=ml_data_pipe)

In [85]:
cv_model.transform(ml_data_pipe).select("label_num", "prediction").show()

+---------+----------+
|label_num|prediction|
+---------+----------+
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      0.0|       0.0|
|      1.0|       1.0|
+---------+----------+
only showing top 20 rows

